In [ ]:
!pip install contractions

In [ ]:
#Dependecies 
import pandas as pd
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from wordcloud import WordCloud
from collections import Counter
from itertools import chain
import contractions
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from nltk.collocations import TrigramAssocMeasures, TrigramCollocationFinder
from nltk import ngrams

In [ ]:
df = pd.read_csv('data/mbti_1.csv')
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
y_label = df.type.unique()
print(y_label)

In [ ]:
label_counts = df['type'].value_counts()

plt.figure(figsize=(20,5))
sns.barplot(label_counts.index, label_counts.values)
plt.ylabel('Counts')
plt.xlabel('Personality Types')
plt.show()

In [ ]:
def clean_data(df, column_name = "posts", new_column="cleaned_post"):
    df[new_column] = df[column_name].apply(lambda x: contractions.fix(x))
    df[new_column] = df[new_column].apply(lambda x: x.lower())
    df[new_column] = df[new_column].apply(lambda x: re.sub(r'@([a-zA-Z0-9_]{1,50})', '', x))
    df[new_column] = df[new_column].apply(lambda x: re.sub(r'#([a-zA-Z0-9_]{1,50})', '', x))
    df[new_column] = df[new_column].apply(lambda x: re.sub(r'http[s]?://\S+', '', x))
    df[new_column]= df[new_column].apply(lambda x: re.sub(r'[^A-Za-z]+', ' ', x))
    df[new_column] = df[new_column].apply(lambda x: re.sub(r' +', ' ', x))
    df[new_column] = df[new_column].apply(lambda x: " ".join([word for word in x.split() if not len(word) <3]))
    return df

cleaned_df = clean_data(df)

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df["words_count"] = cleaned_df["cleaned_post"].apply(lambda x: len(x.split()))
cleaned_df.head(5)

In [ ]:

plt.figure(figsize=(15,10))
sns.swarmplot("type", "words_count", data=df)

In [ ]:
stopword = stopwords.words('english')

In [ ]:
def get_most_frequent(data, stopword, column="cleaned_post", top=25):
    df = data[column].apply(lambda x: " ".join([word for word in x.split() if not word in stopword]))
    counter = Counter(" ".join(df).split())
    return counter.most_common(top)


In [ ]:
most_frequents = get_most_frequent(cleaned_df, stopword)
most_frequents[:20]

In [ ]:
most_frequent_df = pd.DataFrame(most_frequents)   
plt.figure(figsize=(20,5))
sns.barplot(most_frequent_df.iloc[:20, 0], most_frequent_df.iloc[:20, 1])
plt.ylabel('Frequency')
plt.xlabel('Top Frequent Words')
plt.show()

In [ ]:
fig, ax = plt.subplots(len(cleaned_df['type'].unique()), sharex=True, figsize=(5,5*len(cleaned_df['type'].unique())))

k = 0
for i in cleaned_df['type'].unique():
    df = cleaned_df[cleaned_df['type'] == i]
    wordcloud = WordCloud().generate(df['cleaned_post'].to_string())
    ax[k].imshow(wordcloud)
    ax[k].set_title(i)
    ax[k].axis("off")
    k+=1

In [ ]:
def get_ngrams(data, n_gram, new_column, column="cleaned_post"):
    data["tokenized"]  = data[column].apply(lambda x: x.split())
    data["sw_removal"] = data["tokenized"].apply(lambda x: [y for y in x if not y in stopword])
    data[new_column]   = data["sw_removal"].apply(lambda x: list(ngrams(x, n_gram)))
    data.drop(columns  = ["tokenized", "sw_removal"], inplace=True)
    return data

In [ ]:
data = get_ngrams(cleaned_df, n_gram=2, new_column="bigrams")
data.head()

In [ ]:
data = get_ngrams(data, n_gram=3, new_column="trigrams")
data.head()

In [ ]:
X = cleaned_df["cleaned_post"]
X

In [ ]:
y = LabelEncoder().fit_transform(cleaned_df['type'])
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:

vectorizer = TfidfVectorizer(max_features=10000)

In [ ]:
vectorizer.fit(X_train)

In [ ]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
X_train

In [ ]:
tf_idf = pd.DataFrame(X_test.toarray(), columns=vectorizer.get_feature_names_out())
tf_idf.head(10)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost
import pickle
from sklearn import metrics
     

In [ ]:
def create_models():
    nb_clf  = MultinomialNB(alpha=0.01)
    svm_clf = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    dt_clf  = DecisionTreeClassifier(max_depth=7)
    rf_clf  = RandomForestClassifier(n_estimators=750)
    xgb_clf = xgboost.XGBClassifier(use_label_encoder=False)
    return {"NaiveBayes":nb_clf, "SVM":svm_clf, "DecisionTree":dt_clf, "RandomForest":rf_clf, "Xgboost":xgb_clf}

In [ ]:
_metrics = ["Accuracy"]
_columns = ["NaiveBayes", "SVM", "DecisionTree", "RandomForest", "Xgboost"]
     

In [ ]:
evaluation_df = pd.DataFrame(columns=_columns, index=[_metrics])
evaluation_df

In [ ]:
models = create_models()
models

In [ ]:
clf = MultinomialNB().fit(X_train, y_train)
y_predict = clf.predict(X_test)
confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict))
n_right = 0 
for i in range(len(y_predict)):
    if y_predict[i] == y_test[i]:
        n_right+=1
print("Accuracy: %.2f%%"%((n_right/float(len(y_test))*100)))

In [ ]:
clf = SVC(C=1.0, kernel='linear', degree=3, gamma='auto').fit(X_train, y_train)
y_predict = clf.predict(X_test)
confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict))
n_right = 0 
for i in range(len(y_predict)):
    if y_predict[i] == y_test[i]:
        n_right+=1
print("Accuracy: %.2f%%"%((n_right/float(len(y_test))*100)))

In [ ]:
clf = RandomForestClassifier(n_estimators=750).fit(X_train, y_train)
y_predict = clf.predict(X_test)
confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict))
n_right = 0 
for i in range(len(y_predict)):
    if y_predict[i] == y_test[i]:
        n_right+=1
print("Accuracy: %.2f%%"%((n_right/float(len(y_test))*100)))


In [ ]:
clf = xgboost.XGBClassifier().fit(X_train, y_train)
y_predict = clf.predict(X_test)
confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict))
n_right = 0 
for i in range(len(y_predict)):
    if y_predict[i] == y_test[i]:
        n_right+=1
print("Accuracy: %.2f%%"%((n_right/float(len(y_test))*100)))

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train, y_train)
print(f"Training score: {clf.score(X_train, y_train)}")
print(f"Test score: {clf.score(X_test, y_test)}")

In [ ]:
y_predict = clf.predict(X_test)
confusion_matrix(y_test, y_predict)

In [ ]:
print(classification_report(y_test, y_predict))

In [ ]:
n_right = 0 
for i in range(len(y_predict)):
    if y_predict[i] == y_test[i]:
        n_right+=1
print("Accuracy: %.2f%%"%((n_right/float(len(y_test))*100)))